In [1]:
from PIL import Image, ImageFilter
import math
import time
from tqdm import tqdm

In [3]:
from pynput import mouse
top_left = None
def on_click(x, y, button, pressed):
    global top_left
    if not pressed:
        # Stop listener
        top_left = (x,y)
        return False
print("Click in the top left of Giga Manga Canvas")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print('Top Left',top_left)

bottom_right = None
def on_click(x, y, button, pressed):
    global bottom_right
    if not pressed:
        # Stop listener
        bottom_right = (x,y)
        return False
print("Click in the bottom right of Giga Manga Canvas")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print("Bottom Right", bottom_right)
s_width =  bottom_right[0]- top_left[0]
s_height = bottom_right[1]- top_left[1]
print('Selected Width', s_width)
print('Selected Height', s_height)

Click in the top left of Giga Manga Canvas
Top Left (1780, 322)
Click in the bottom right of Giga Manga Canvas
Bottom Right (2298, 841)
Selected Width 518
Selected Height 519


In [25]:
fc_image = Image.open("porygon.png")
image = fc_image.convert("L")
image = image.filter(ImageFilter.FIND_EDGES)
image.save(r"Edge_Sample.png")

In [26]:
w_padding = 0
h_padding = 0
if s_width > image.width:
    print('image width smaller than canvas, centering')
    w_padding = (s_width-image.width) /2
else:
    print("Your image is larger than your selected width")
if s_height > image.height:
    print('image height smaller than canvas, centering')
    h_padding = (s_width-image.width) /2
else:
    print("Your image is larger than your selected height")

print('w_padding',math.floor(w_padding))
print('h_padding',math.floor(h_padding))

image width smaller than canvas, centering
image height smaller than canvas, centering
w_padding 3
h_padding 3


In [27]:
from pynput.mouse import Button, Controller
mouse = Controller() 

In [28]:
class Line():
    def __init__(self,xy1,xy2):
        self.xy1 = xy1
        self.xy2 = xy2
        
    def drag (self):
        mouse.position = self.xy1
        mouse.press(Button.left)
        mouse.position = self.xy2
        mouse.release(Button.left)

In [29]:
class lineFinder():
    def __init__(self,image,resolution):
        self.image = image
        self.lines = []
        self.resolution =resolution
        
        
    def addLine(self,row):
        xy1 = min(row)
        xy2 = max(row)
        xy1 = ( xy1[0]+top_left[0]+w_padding , xy1[1]+top_left[1]+h_padding)
        xy2 = ( xy2[0]+top_left[0]+w_padding , xy2[1]+top_left[1]+h_padding)
        self.lines.append(Line(xy1,xy2))
        
    def detect(self):
        inRow=False
        image = self.image
        for y in range(0,image.height,self.resolution):
            for x in range(0,image.width,self.resolution):
                pix = image.getpixel((x,y))
                if pix > 1:
                    if inRow:
                        row.append((x,y))
                    else:
                        inRow=True
                        row = []
                else:
                    if len(row)>1 and len(row) < 100:
                        self.addLine(row)
                    inRow=False
                    row = []
            if len(row)>1 and len(row) < 100:
                self.addLine(row)
                inRow=False
                row = []
                
                
        
        inRow=False
        for x in range(0,image.width,self.resolution):
            for y in range(0,image.height,self.resolution):
                pix = image.getpixel((x,y))
                if pix > 1:
                    if inRow:
                        row.append((x,y))
                    else:
                        inRow=True
                        row = []
                else:
                    if len(row)>1 and len(row) < 100:
                        self.addLine(row)
                    inRow=False
                    row = []
            if len(row)>1 and len(row) < 100:
                self.addLine(row)
                inRow=False
                row = []

In [30]:
a = lineFinder(image,1)

In [31]:
a.detect()

In [32]:
for c in tqdm(a.lines):
    c.drag()
    time.sleep(0.02)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5038/5038 [02:02<00:00, 41.05it/s]
